<a href="https://colab.research.google.com/github/maricrismaros/InteligenciaArtificialIMTC_1678836/blob/main/PIA_IA_y_RN_Verano2021_1678836.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install Kaggle
!pip install -q kaggle 

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"maricris","key":"699ee5379e9c28ac8b7eea29b1a5f4dc"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

401 - Unauthorized


In [ ]:
! kaggle datasets download -d puneet6060/intel-image-classification

401 - Unauthorized


In [ ]:
!unzip intel-image-classification.zip

unzip:  cannot find or open intel-image-classification.zip, intel-image-classification.zip.zip or intel-image-classification.zip.ZIP.


CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os
import PIL
import PIL.Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

from os import listdir
from os.path import isfile, join

import cv2
from tqdm.notebook import tqdm
from scipy import ndimage

DOWNLOAD AND PREPARE THE DATASHEET

In [ ]:
path_train = '/content/seg_train/seg_train'
path_build = '/buildings/'
path_forest = '/forest/'
path_glacier = '/glacier/'
path_mountain = '/mountain/'
path_sea = '/sea/'
path_street = '/street/'
path_test = '/content/seg_test/seg_test'

In [ ]:
class_label = ['Building', 'Forest', 'Glacier', 'Mountain', 'Sea', 'Street']

In [ ]:
paths = [path_build, path_forest, path_glacier, path_mountain, path_sea, path_street]
paths

In [ ]:
data = [] #Creacion de una variable del tipo objeto o Array
labels = [] #Creacion de una variable del tipo objeto o Array
IMG_SIZE = 150
augmentation = 10
for i, path in enumerate(paths): #Recorrido del arreglo Paths 
    img_urls = listdir(path_train+path) #Busqueda de un directorio de manera que entra a cierta carpeta para su recorruido etiqueta la carpeta de edificios pór ejemplo con la etiqueta building

    print(f"Importing {class_label[i]}s")
    for img_name in tqdm(img_urls):
        img = cv2.imread(path_train+path+img_name)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        #for u in np.linspace(-5,5,num= augmentation):
        #    im = ndimage.rotate(img, u)
        #    im = cv2.resize(im,(IMG_SIZE,IMG_SIZE))
        im = np.array(img)
        data.append(im)  ## esta linea empuja un dato a la casilla del array Data
        labels.append(i)
        im = np.array(cv2.flip(img,1))
        data.append(im)
        labels.append(i)

In [ ]:
data = np.array(data) # Transforma labels en un array 
data.shape

In [ ]:
labels = np.array(labels)
labels.shape # Transforma labels en un array 

VERIFY THE DATA

In [ ]:
plt.figure(figsize=(10,10))    
for i in range(25):      
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data[i]) 
    plt.xlabel(class_label[labels[i]])
plt.show()

BASE CONVOLUCIONAL

In [ ]:
ini = tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=1e-2, seed=None
)

model = Sequential()
model.add(Conv2D(32,
                 (3,3),
                 activation='relu',
                 kernel_initializer = ini,
                 input_shape=(IMG_SIZE,IMG_SIZE,3)
                )
         )
model.add(Conv2D(32,
                 (3,3),
                 activation='relu',
                 kernel_initializer = ini,
                )
         )
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64,
                 (3,3),
                 activation='relu',
                 kernel_initializer = ini,
                )
         )
model.add(Dropout(0.3))

model.add(Conv2D(64,
                 (3,3),
                 activation='relu',
                 kernel_initializer = ini,
                )
         )
model.add(MaxPool2D((2,2)))

model.add(Dense(64,activation='relu'))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(6, activation= 'sigmoid'))


optim = tf.keras.optimizers.Adam(
    learning_rate=5e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-07
)

model.compile(
    optimizer=optim, loss='sparse_categorical_crossentropy', metrics=['accuracy']
)

model.summary()

COMPILE THE TRAIN MODEL 

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=50
)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size = 0.2, shuffle= True)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, restore_best_weights=True
)


epochs = 10
batch_size = 16

history = model.fit(
    x_train,y_train, batch_size=batch_size, epochs=epochs,
    callbacks=[early_stop], validation_split=0.2, shuffle=True
)

In [ ]:
history.history.keys()

EVALUATE THE MODEL 

In [ ]:
plt.figure(figsize= (10,5))
plt.plot(range(len(history.history['loss'])),history.history['loss'], label= 'Loss')
plt.plot(range(len(history.history['val_loss'])),history.history['val_loss'], label= 'Val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize= (10,5))
plt.plot(range(len(history.history['accuracy'])),history.history['accuracy'], label= 'Accuracy')
plt.plot(range(len(history.history['val_accuracy'])),history.history['val_accuracy'], label= 'Val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Acuracy vs Epoch')
plt.legend()
plt.grid()
plt.show()

max_ac = np.max(history.history['val_accuracy'])
print(f'Max Accuracy was: {max_ac}')